# Data Collection: SQLite Database

## Import modules

In [1]:
import numpy as np
import pandas as pd
import sqlite3

## Prepare Data

In [2]:
groups = ["Family", "Friend", "Colleague", "Neighbor"]

people = [
    (1, 1, "Anna", 42, 45850.75), (2, 1, "Peter", 40, 42500), (3, 1, "Linda", 45, 55500.50),
    (4, 2, "Mark", 47, 35800.00), (5, 2, "Steffi", 38, 51367), (6, 2, "Lars", 50, 57500.50), 
    (7, 3, "Bernd", 50, 48000.00), (8, 3, "Petra", 43, 57350), (9, 3, "Sabine", 49, 47200),
    (10, 4, "Lotte", 41, 45500), (11, 4, "Gunter", 60, 35400), (12, 4, "Rene", 38, 51500),
]

## Connect to database and create a cursor object

In [3]:
#con = sqlite3.connect("my_database.db")  # Creates a file
# or
con = sqlite3.connect(":memory:")  # is only in memory and nor persistent
cursor = con.cursor()  # create a cursor object

## Create tables

In [6]:
# create table people
sql = """CREATE TABLE IF NOT EXISTS people (
  id INTEGER PRIMARY KEY, 
  group_id INTEGER NOT NULL,
  name TEXT,
  age INTEGER,
  salery REAL
)"""
cursor.execute(sql)

# create table group
sql = """CREATE TABLE IF NOT EXISTS groups (
  id INTEGER PRIMARY KEY,
  name TEXT
)"""
cursor.execute(sql)

## List all tables in database

In [11]:
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
tables

[('people',), ('groups',)]

## Insert some Data

In [12]:
# insert groups
for group in enumerate(groups, 1):
    cursor.execute("INSERT INTO groups VALUES(?,?)", group)

# insert people
for person in people:
    cursor.execute("INSERT INTO people VALUES(?,?,?,?,?)", person)

## Query all people

In [13]:
df = pd.read_sql_query(f"SELECT * FROM people", con)
df

,id,group_id,name,age,salery
0,1,1,Anna,42,45850.75
1,2,1,Peter,40,42500.00
2,3,1,Linda,45,55500.50
3,4,2,Mark,47,35800.00
4,5,2,Steffi,38,51367.00
5,6,2,Lars,50,57500.50
6,7,3,Bernd,50,48000.00
7,8,3,Petra,43,57350.00
8,9,3,Sabine,49,47200.00
9,10,4,Lotte,41,45500.00


## Joined query

In [15]:
sql = """
SELECT *
  FROM people 
  JOIN groups 
    ON people.group_id=groups.id"""

df_joined = pd.read_sql_query(sql, con)
df_joined

,id,group_id,name,age,salery,id,name
0,1,1,Anna,42,45850.75,1,Family
1,2,1,Peter,40,42500.00,1,Family
2,3,1,Linda,45,55500.50,1,Family
3,4,2,Mark,47,35800.00,2,Friend
4,5,2,Steffi,38,51367.00,2,Friend
5,6,2,Lars,50,57500.50,2,Friend
6,7,3,Bernd,50,48000.00,3,Colleague
7,8,3,Petra,43,57350.00,3,Colleague
8,9,3,Sabine,49,47200.00,3,Colleague
9,10,4,Lotte,41,45500.00,4,Neighbor


In [16]:
con.close()